# Setup
## Imports

In [1]:
from __future__ import print_function

import os
import json
import numpy as np
import pandas as pd
import plotly.tools as tls
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from itertools import chain
from IPython.display import clear_output
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline

plt.style.use("ggplot")

## DACA
### Pull DACA related tweets.

In [2]:
# -- List data files.
fnames = os.listdir("../data")
# -- List of words that will identify DACA related tweets.
daca_words = ["daca", "dreamers"]

In [3]:
daca_sentiment = []
# -- For each representative...
for ii, fname in enumerate(fnames):
    # -- Print Status
    print("DACA: {} ({}/{})                                 " \
          .format(fname[:-5], ii+1, len(fnames)))
    clear_output(wait=True)
    # -- Load data file.
    rep = json.load(open(os.path.join("..", "data", fname), "r"))
    # -- If there are more than 100 tweets.
    if len(rep["tweets"]) > 100:
        daca_sent = []
        # -- Parse tweets that mention DACA or dreamers.
        for tweet in rep["tweets"]:
            if any(word in tweet["full_text"].lower() for word in daca_words):
                daca_sent.append(tweet["analysis"]["sentiment"]["compound"])
        if rep["twitter_account"] == "realDonaldTrump":
            name = "Donald Trump"
        elif rep["twitter_account"] == "mike_pence":
            name = "Mike Pence"
        else:
            name = rep["first_name"] + " " + rep["last_name"]
        # -- Append representative data.
        daca_sentiment.append([
                name,
                rep["twitter_account"],
                rep.get("party", "NA"),
                rep.get("state", "NA"),
                len(rep["tweets"]),
                daca_sent])

DACA: RepHastingsFL (532/532)                                 


### Create plotting attributes

In [4]:
# -- Text format for plotly.
text = ("<b>{} ({}-{})</b><br><i>@{}</i><br>" +
        "<b>{:.2f}% ({} / {})</b>")
# -- Colors for plotly.
dcolor = "rgb({},{},{})".format(*cm.get_cmap("Blues")(0.6))
rcolor = "rgb({},{},{})".format(*cm.get_cmap("Reds")(0.6))
ecolor = "rgb({},{},{})".format(255, 165, 0)
icolor = "rgb({},{},{})".format(*cm.get_cmap("gist_gray_r")(0.6))

In [5]:
# -- Columns names for daca_sentiment.
cols = ["full_name", "twitter_account", "party", "state", "ntweets", "daca_sent"]
df = pd.DataFrame(daca_sentiment, columns=cols)
# -- Count of daca tweets.
df["ndaca_tweets"] = [len(ll) for ll in df.daca_sent]
# -- Count of daca tweets per 100.
df["ndaca_percent"] = 100. * df.ndaca_tweets / df.ntweets
# -- Sort values by daca tweet prop. and party.
df.sort_values(["ndaca_percent", "full_name"], 
               ascending=[False, True], inplace=True)
df.reset_index(inplace=True, drop=True)
# -- Set empty values.
xx = []
yy = []
cc = []
txt = []
ry = 0
rx = 0
# -- For each representative in df.
for idx, rep in df.iterrows():
    # -- Create plotly text.
    txt.append(text.format(rep.full_name.encode('utf-8'), rep.party, rep.state,
                           rep.twitter_account, rep.ndaca_percent,
                           rep.ndaca_tweets, rep.ntweets))
    # -- Add color based on party affiliation.
    if rep["party"] == "D":
        color = dcolor
    elif rep["party"] == "R":
        color = rcolor
    elif rep["party"] == "NA":
        color = ecolor
    elif rep["party"] == "I":
        color = icolor
    cc.append(color)
    # -- Save xx yy for plotly plot.
    xx.append(rx)
    yy.append(ry)
    rx += 1
    if ((idx + 1) % 20) == 0:
        rx = 0
        ry -= 1
# -- Create columns in df.
df["colors"] = cc
df["xx"] = xx
df["yy"] = yy
df["text"] = txt
df.head(1)

,full_name,twitter_account,party,state,ntweets,daca_sent,ndaca_tweets,ndaca_percent,colors,xx,yy,text
0,Richard Durbin,SenatorDurbin,D,IL,3247,"[-0.6124, 0.0772, -0.128, 0.8298, 0.4939, 0.0,...",408,12.565445,"rgb(0.290980392157,0.594509803922,0.789019607843)",0,0,<b>Richard Durbin (D-IL)</b><br><i>@SenatorDur...


### Plotly figure

In [8]:
# -- Constants.
size = 1
data = []
shapes = []
annotations = []
# -- For each party create trace.
for party in ["D", "R", "NA", "I"]:
    trace = go.Scatter(
        x=df[df.party == party].xx,
        y=df[df.party == party].yy,
        mode="markers",
        marker=dict(size=size, color=df[df.party == party].colors),
        text=df[df.party == party].text,
        hoverinfo="text",
    )
    data.append(trace)
# -- Create shapes and annotations.
for idx, rep in df.iterrows():
    shape = {"type": "rect",
             "x0": rep.xx - 0.4,
             "y0": rep.yy - 0.4,
             "x1": rep.xx + 0.4,
             "y1": rep.yy + 0.4,
             "line": {"width": 0},
             "fillcolor": rep.colors}
    if rep.ndaca_percent == 0:
        anno = "0"
    elif rep.ndaca_percent < 0.95:
        anno = "{:.1f}".format(rep.ndaca_percent)[1:]
        
    else:
        anno = "{:.0f}".format(rep.ndaca_percent)
    annotation = {"x": rep.xx,
                  "y": rep.yy,
                  "xref": "x",
                  "yref": "y",
                  "text": "<b>{}</b>".format(anno),
                  "showarrow": False,
                  "align": "center",
                  "font": dict(color="rgb(255, 255, 255)",
                               family="arial")}
    shapes.append(shape)
    annotations.append(annotation)
less = {"x": -.75,
        "y": df.yy.min() + 0.5,
        "textangle": -90,
        "xref": "x",
        "yref": "y",
        "text": "<i>← Lower</i>",
        "showarrow": False,
        "font": dict(color="rgb(160, 160, 160)", family="arial", size=14)}
more = {"x": -.75,
        "y": df.yy.max() - 0.5,
        "textangle": -90,
        "xref": "x",
        "yref": "y",
        "text": "<i>Higher →</i>",
        "showarrow": False,
        "font": dict(color="rgb(160, 160, 160)", family="arial", size=14)}
title = {"x": df.xx.max() / 2.,
         "y": df.yy.max() + 2.5,
         "xref": "x",
         "yref": "y",
         "text": "<b>Percent of Congress Members'<br>Tweets That Relate to DACA</b>",
         "showarrow": False,
         "font": dict(color="rgb(102, 102, 102)", 
                      family="arial", size=32)}
note = {"x": df.xx.max() + 0.5,
        "y": df.yy.min(),
        "xanchor": "right",
        "yanchor": "center",
        "xref": "x",
        "yref": "y",
        "align": "right",
        "text": "<i>Tweets collected alphabetically by username on 2018/01/15 - 2018/01/17.<br>Tweets relate to DACA if they include 'daca' or 'dreamers'.</i>",
        "showarrow": False,
        "font": dict(color="rgb(160, 160, 160)", family="arial", size=12)}
for annotation in [less, more, title, note]:
    annotations.append(annotation)
# -- Create plot layout.
layout= go.Layout(
    hovermode="closest",
    height=1300,
    xaxis= dict(
        zeroline= False,
        showgrid=False,
        showticklabels=False
    ),
    yaxis=dict(
        zeroline= False,
        showgrid=False,
        showticklabels=False
    ),
    showlegend= False,
    shapes=shapes,
    annotations=annotations
)
# --
fig = go.Figure(data=data, layout=layout)
iplot(fig)